In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2    
import os, glob, re
import zipfile as zf
import WIDParseTools as wpt
import pandas as pd
import time

<H2>Enter report details here</H2>

In [ ]:
# Path where the wid file is
mypath = 'C:\\wids\\'
# part of wid file name
RapportCode = 'blah'

<H3>Analyse van individueel rapport -> Excel </H3>

In [ ]:
%%time


allvardepslist = []

# showalldeps = True: haalt alle links op van variabele naar het rapport
# showalldeps = False: haalt alleen het korste pad op van variabele naar rapport
showalldeps = True

widfiles = mypath +  '*.wid'
for widfile in glob.glob(widfiles):

    widfilearchive = zf.ZipFile(widfile, mode='r')

    docprops = wpt.ReadDocumentProps(widfilearchive)
   
    reportName = docprops['name']

    if reportName.find(RapportCode)>-1:
    
        print(widfile,':',reportName)
        # print(docprops)
    
        df_vardependencies, df_varreportusage, df_alerters = wpt.AnalyzeReport(widfilearchive, showalldeps)
        df_vardependencies.insert(0, 'ReportName',  reportName, True)
    
        df_formulas = wpt.GetReportFormulas(widfilearchive)
    
        widfilearchive.close()
        
        outputdir = mypath
        # Write results to excel
        exportfilename  = outputdir + reportName + '.xlsx'
        print('writing to ', exportfilename)
        with pd.ExcelWriter(outputdir + reportName + '.xlsx') as excel_writer:
            df_vardependencies.to_excel(excel_writer, sheet_name='Variable dependencies', index=False)
            df_varreportusage.to_excel(excel_writer, sheet_name='Variable report usage', index=False)
            df_formulas.to_excel(excel_writer, sheet_name='Report formulas', index=False)
            df_alerters.to_excel(excel_writer, sheet_name='Alerter status', index=False)

    widfilearchive.close()